In [1]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Activation, Dropout,Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Lambda, Input, AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import keras
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.applications import ResNet50

2023-06-12 17:55:25.724772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [3]:
images_1_path = '/Users/baraajadallah/Repos/Pediatric Chest X-ray Pneumonia/train/Normal'
images_2_path = '/Users/baraajadallah/Repos/Pediatric Chest X-ray Pneumonia/train/PNEUMONIA'
images_3_path = '/Users/baraajadallah/Repos/Pediatric Chest X-ray Pneumonia/test/NORMAL'
images_4_path = '/Users/baraajadallah/Repos/Pediatric Chest X-ray Pneumonia/test/PNEUMONIA'

In [4]:
images_1 = load_images_from_folder(images_1_path)
images_2 = load_images_from_folder(images_2_path)
images_3 = load_images_from_folder(images_3_path)
images_4 = load_images_from_folder(images_4_path)

In [5]:
df_1 = pd.DataFrame()
df_2 = pd.DataFrame()
df_3 = pd.DataFrame()
df_4 = pd.DataFrame()

In [6]:
df_1['images'] = images_1
df_1['tumor'] = 0

df_2['images'] = images_2
df_2['tumor'] = 1

df_3['images'] = images_3
df_3['tumor'] = 0

df_4['images'] = images_4
df_4['tumor'] = 1

In [7]:
df = pd.concat([df_1,df_2, df_3, df_4], ignore_index=True)

df = df.sample(frac = 1)

In [8]:
df.head()

,images,tumor
4954,"[[[181, 181, 181], [180, 180, 180], [182, 182,...",1
1938,"[[[44, 44, 44], [47, 47, 47], [49, 49, 49], [5...",1
4241,"[[[0, 0, 0], [8, 8, 8], [5, 5, 5], [0, 0, 0], ...",1
4724,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",1
5792,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",1


In [ ]:
def extract_gabor_filters(image):

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Define the parameters for Gabor filters
    ksize = 5  # Size of the Gabor filter kernel
    sigma = 4.0  # Standard deviation of the Gaussian function
    theta = np.pi / 4  # Orientation of the Gabor filter
    lambd = 10.0  # Wavelength of the sinusoidal factor
    gamma = 0.5  # Spatial aspect ratio

    # Create the Gabor filter
    gabor_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma)
    
    # Apply the Gabor filter to the grayscale image
    filtered_image = cv2.filter2D(gray, cv2.CV_64F, gabor_kernel)

    return filtered_image

#df['images'] = df['images'].apply(lambda x: cv2.imread(x))

# Apply Gabor filters to the images
df['gabor_filtered'] = df['images'].apply(extract_gabor_filters)
